# ABBA test - 전력 사용량 예측

## import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from datetime import datetime

import sys
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings(action='ignore')
from matplotlib import font_manager,rc
rc('font', family='AppleGothic')

## 데이터

### 데이터 불러오기

#### building_info 데이터 불러오기

In [2]:
# building_info 데이터 불러오기
building_info = pd.read_csv('input/building_info.csv')
display(building_info.head())
print(building_info.info())

#### train_df 데이터 불러오기

In [3]:
# train_df 데이터 불러오기
train_df = pd.read_csv('input/train.csv')
display(train_df.head())
print(train_df.info())

#### test_df 데이터 불러오기

In [4]:
# test_df 데이터 불러오기
test_df = pd.read_csv('input/test.csv')
display(test_df.head())
print(test_df.info())

### building info
- 100개 건물 정보
- 건물 번호, 건물 유형, 연면적, 냉방 면적, 태양광 용량, ESS 저장 용량, PCS 용량

#### column명 변환
- building_info의 column명을 편의를 위해 영어로 대체

In [5]:
# building_info의 column명을 편의를 위해 영어로 대체
building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

building_info.head()

####  building_type 변환
- building_info의 building_type column을 편의를 위해 영어로 대체

In [6]:
# building_info의 building_type column을 편의를 위해 영어로 대체
translation_dict = {
    '건물기타': 'Other_Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data_Center',
    '백화점및아울렛': 'Department_Store_and_Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research_Institute',
    '지식산업센터': 'Knowledge_Industry_Center',
    '할인마트': 'Discount_Mart',
    '호텔및리조트': 'Hotel_and_Resort'
}
building_info['building_type'] = building_info['building_type'].replace(translation_dict)
building_info.head()

#### 결측치 처리 
- solar_power_capacity, ess_capacity, pcs_capacity 컬럼의 '-' 로 되어있는 결측치를 0으로 바꿔주고 object로 되어있는 type을 float64로 변환

In [7]:
# solar_power_capacity, ess_capacity, pcs_capacity 컬럼의 '-' 로 되어있는 결측치를 0으로 바꿔주고 object로 되어있는 type을 float64로 변환
building_info = building_info.replace('-',0)
building_info[['solar_power_capacity','ess_capacity','pcs_capacity']] = building_info[['solar_power_capacity','ess_capacity','pcs_capacity']].astype('float64')
display(building_info.head())
print(building_info.info())

#### Data Cleaning

In [8]:
# 20번 건물이 다른 건물들에 비해서 너무 큰 total_area 값임을 확인
display(building_info[building_info.building_type=='Public'])

plt.figure(figsize=(10, 3))
plt.bar(building_info.building_number,building_info.total_area)
plt.title('건물 연면적 비교')
plt.xlabel('building_number')
plt.ylabel('total_area(m2)')
plt.show()

plt.figure(figsize=(10, 3))
plt.bar(building_info.building_number, building_info.cooling_area)
plt.title('건물 연면적 비교')
plt.xlabel('building_number')
plt.ylabel('cooling_area(m2)')
plt.show()

temp = building_info[building_info['building_type']=='Public']
temp_drop_20 = temp.drop(index=19)
temp_drop_20.total_area.mean()

print(f"20번 건물을 제외한 Public 건물 연면적 평균: {temp_drop_20['total_area'].mean()} (m2)")
print(f"20번 건물의 연면적: {temp.loc[19,'total_area']} (m2)")
print(f"20번 건물의 연면적은 Public 건물 평균의 {(temp.loc[19,'total_area']/temp_drop_20['total_area'].mean())}배 이다.")
print()
print(f"20번 건물을 제외한 Public 건물 냉방면적 평균: {temp_drop_20['cooling_area'].mean()} (m2)")
print(f"20번 건물의 냉방면적: {temp.loc[19,'cooling_area']} (m2)")
print(f"20번 건물의 냉방면적은 Public 건물 평균의 {(temp.loc[19,'cooling_area']/temp_drop_20['cooling_area'].mean())}배 이다.")

In [9]:
building_info.loc[19,'total_area'] = round(temp_drop_20['total_area'].mean()*1.2, 2)
building_info.loc[19,'cooling_area'] = round(temp_drop_20['cooling_area'].mean()*1.2, 2)
display(building_info[building_info.building_type=='Public'])
building_info.groupby('building_type').get_group('Public').total_area.mean()

### train_df
- train 데이터 : 100개 건물들의 2022년 06월 01일부터 2022년 08월 24일까지의 데이터
- 일시별 기온, 강수량, 풍속, 습도, 일조, 일사 정보 포함
- 전력사용량(kWh) 포함

#### column명 변환
- train_df의 column명을 편의를 위해 영어로 대체

In [10]:
# train_df의 column명을 편의를 위해 영어로 대체
train_df = train_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})

#### date_time 형식 변경 및 분활

In [11]:
# 형식 변경: 20220601 00 -> 2022-06-01 00:00:00
# object -> datetime64
train_df['date_time'] = pd.to_datetime(train_df['date_time'], format='%Y%m%d %H')

# date time feature 생성
train_df['year'] = train_df['date_time'].dt.year
train_df['month'] = train_df['date_time'].dt.month
train_df['day'] = train_df['date_time'].dt.day
train_df['hour'] = train_df['date_time'].dt.hour
train_df['day_of_week'] = train_df['date_time'].dt.dayofweek
train_df['day_of_year'] = train_df['date_time'].dt.dayofyear
train_df.head()

#### 결측치 처리

In [12]:
# 결측치 확인
print(train_df.shape)
train_df.isnull().sum()

In [13]:
# rainfall경우 204000행 중 약 80%인 160069행이 결측되었지만 우리나라의 비가 오는날이 안 오는늘의 80% 정도라 생각하여 0으로 채움
train_df.fillna({"rainfall": 0}, inplace=True)
train_df.isnull().sum()

In [14]:
# 결측치가 적은 windspeed, humidity 컬럼의 결측치릐 선행값으로 대체
display(train_df[~(train_df['windspeed'] >= 0)])
display(train_df[~(train_df['humidity'] >= 0)])
train_df['windspeed'].fillna(method='ffill', axis=0, inplace=True)
train_df['humidity'].fillna(method='ffill', axis=0, inplace=True)
train_df.isnull().sum()

In [15]:
# sunshine, solar_radiation 컬름의 결측치는 밤의 경우 일조량이 없음으로 0으로 채움
train_df.fillna({"sunshine": 0, 'solar_radiation':0}, inplace=True)
train_df.isnull().sum()

#### Data Cleaning

In [16]:
# temperature, windspeed, humidity의 값들이 정상 범위 내에 있음을 확인
for i in ['temperature', 'windspeed', 'humidity']:
    print(f"{i}의 최대값: {max(train_df[i])}")
    print(f"{i}의 평균값: {np.mean(train_df[i])}")
    print(f"{i}의 중위값: {np.median(train_df[i])}")
    print(f"{i}의 최소값: {min(train_df[i])}")
    print()

### test_df
- test 데이터 : 100개 건물들의 2022년 08월 25일부터 2022년 08월 31일까지의 데이터
- 일시별 기온, 강수량, 풍속, 습도의 예보 정보

#### column명 변환
- test_df의 column명을 편의를 위해 영어로 대체

In [17]:
# test_df의 column명을 편의를 위해 영어로 대체
test_df = test_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})

#### date_time 형식 변경 및 분활

In [18]:
# 형식 변경: 20220601 00 -> 2022-06-01 00:00:00
# object -> datetime64
test_df['date_time'] = pd.to_datetime(test_df['date_time'], format='%Y%m%d %H')

# date time feature 생성
test_df['year'] = test_df['date_time'].dt.year
test_df['month'] = test_df['date_time'].dt.month
test_df['day'] = test_df['date_time'].dt.day
test_df['hour'] = test_df['date_time'].dt.hour
test_df['day_of_week'] = test_df['date_time'].dt.dayofweek
test_df['day_of_year'] = test_df['date_time'].dt.dayofyear
test_df.head()

#### 결측치 처리

In [19]:
# 결측치 확인
print(test_df.shape)
test_df.isnull().sum()

#### Data Cleaning

In [20]:
# temperature, windspeed, humidity의 값들이 정상 범위 내에 있음을 확인
for i in ['temperature', 'windspeed', 'humidity']:
    print(f"{i}의 최대값: {max(test_df[i])}")
    print(f"{i}의 평균값: {np.mean(test_df[i])}")
    print(f"{i}의 중위값: {np.median(test_df[i])}")
    print(f"{i}의 최소값: {min(test_df[i])}")
    print()

### Feature Engineering

#### 체감온도(WCT) feature 생성
- 체감온도 (Wind Chill Temperature): 기온과 풍속을 고려하여 인체에 느껴지는 온도를 나타내는 지표입니다. 높은 바람이 불 때, 실제 기온보다 더 춥게 느껴질 수 있습니다.

In [21]:
train_df['WCT'] = 13.12 + 0.6125*train_df['temperature'] - 11.37*(train_df['windspeed'] ** 0.16) + 0.3965*(train_df['windspeed']**0.16)*train_df['temperature']
train_df.head()

In [22]:
test_df['WCT'] = 13.12 + 0.6125*test_df['temperature'] - 11.37*(test_df['windspeed'] ** 0.16) + 0.3965*(test_df['windspeed']**0.16)*test_df['temperature']
test_df.head()

#### 불쾌 지수(THI) feature 생성
- 불쾌 지수(습도 지수) (Temperature Humidity Index): 습도와 온도를 고려하여 불쾌한 정도를 표현하는 지표입니다. 높은 습도와 높은 기온은 불쾌함을 증가시킬 수 있습니다.

In [23]:
train_df['THI'] = 9/5*train_df['temperature'] - 0.55 * (1-train_df['humidity']/100)*(9/5*train_df['humidity']-26)+32
train_df.head()

In [24]:
test_df['THI'] = 9/5*test_df['temperature'] - 0.55 * (1-test_df['humidity']/100)*(9/5*test_df['humidity']-26)+32
test_df.head()

#### 휴일(holiday) feature 생성
>'건물기타': 'Other_Buildings'\
'공공': 'Public' - 주말 전력 소비 감소\
'대학교': 'University' - 주말 전력 소비 감소\
'데이터센터': 'Data_Center' - 매일 전력 소비 일정\
'백화점및아울렛': 'Department_Store_and_Outlet' - 월요일 전력 소비 감소\
'병원': 'Hospital' - 주말 전력 소비 감소\
'상용': 'Commercial' - 주말 전력 소비 감소\
'아파트': 'Apartment' - 매일 전력 소비 일정\
'연구소': 'Research_Institute' - 주말 전력 소비 감소\
'지식산업센터': 'Knowledge_Industry_Center' - 주말 전력 소비 감소\
'할인마트': 'Discount_Mart' - 일요일 전력 소비 감소\
'호텔및리조트': 'Hotel_and_Resort' - 매일 전력 소비 일정

In [25]:
weekend = ['2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19',
           '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-09', '2022-07-10',
           '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31',
           '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21',
           '2022-08-27', '2022-08-28']
holiday = ['2022-06-01', '2022-06-06', '2022-08-15'] #지방선거일, 현충일, 광복절

In [26]:
# 건물정보 중 건물유형 병합
train_df = train_df.merge(building_info[['building_number', 'building_type']], on='building_number')
test_df = test_df.merge(building_info[['building_number', 'building_type']], on='building_number')

In [27]:
# 주말과 휴일을 모두 쉬는 경우
train_df['holiday'] = 0

wh = train_df.building_type.isin(['Other_Buildings', 'Public', 'University', 'Hospital', 'Commercial', 'Research_Institute', 'Knowledge_Industry_Center'])
train_df['holiday'] = train_df[wh]['date_time'].dt.date.astype('str').isin(weekend+holiday)

# 매일 전력소비 일정
allday = train_df.building_type.isin(['Data_Center', 'Apartment', 'Hotel_and_Resort'])
train_df['holiday'][allday] = False

# 월요일 전력소비 감소
train_df['holiday'][(train_df.building_type == 'Department_Store_and_Outlet') & (train_df['day_of_week']==0)] = True
train_df['holiday'][(train_df.building_type == 'Department_Store_and_Outlet') & (train_df['day_of_week']!=0)] = False

# 일요일 전력소비 감소
train_df['holiday'][(train_df.building_type == 'Discount_Mart') & (train_df['day_of_week']==6)] = True
train_df['holiday'][(train_df.building_type == 'Discount_Mart') & (train_df['day_of_week']!=6)] = False

train_df.pop('building_type')
train_df = train_df.replace({'holiday':{False: 0, True: 1}})
train_df[train_df['holiday']==1].head()

In [28]:
# 주말과 휴일을 모두 쉬는 경우
test_df['holiday'] = 0

wh = test_df.building_type.isin(['Other_Buildings', 'Public', 'University', 'Hospital', 'Commercial', 'Research_Institute', 'Knowledge_Industry_Center'])
test_df['holiday'] = test_df[wh]['date_time'].dt.date.astype('str').isin(weekend+holiday)

# 매일 전력소비 일정
allday = test_df.building_type.isin(['Data_Center', 'Apartment', 'Hotel_and_Resort'])
test_df['holiday'][allday] = False

# 월요일 전력소비 감소
test_df['holiday'][(test_df.building_type == 'Department_Store_and_Outlet') & (test_df['day_of_week']==0)] = True
test_df['holiday'][(test_df.building_type == 'Department_Store_and_Outlet') & (test_df['day_of_week']!=0)] = False

# 일요일 전력소비 감소
test_df['holiday'][(test_df.building_type == 'Discount_Mart') & (test_df['day_of_week']==6)] = True
test_df['holiday'][(test_df.building_type == 'Discount_Mart') & (test_df['day_of_week']!=6)] = False

test_df.pop('building_type')
test_df = test_df.replace({'holiday':{False: 0, True: 1}})
test_df[test_df['holiday']==1].head()

### 데이터 저장(for ML, DL)

In [29]:
train_df_dl = train_df.copy()
test_df_dl = test_df.copy()

# train_df와 test_df에 건물정보를 병합한 후 train_df와 test_df의 features을 동일하게 구성함 
print('building_info')
display(building_info.head())
display(building_info.dtypes)
print('-'*150)
print('train_df_dl')
train_df_dl = train_df_dl[['num_date_time', 'building_number', 'date_time', 'temperature',
       'rainfall', 'windspeed', 'humidity', 'year', 'month', 'day',
       'hour', 'day_of_week', 'day_of_year', 'WCT', 'THI', 'holiday', 'power_consumption']]
train_df_dl = pd.merge(train_df_dl, building_info, on='building_number', how='left')
train_df_dl = train_df_dl.drop(columns=['solar_power_capacity', 'ess_capacity', 'pcs_capacity'])
temp = train_df_dl.pop('power_consumption')
train_df_dl['power_consumption'] = temp
display(train_df_dl.head())
display(train_df_dl.dtypes)
print('-'*150)
print('test_df_dl')
test_df_dl = test_df_dl[['num_date_time', 'building_number', 'date_time', 'temperature',
       'rainfall', 'windspeed', 'humidity', 'year', 'month', 'day',
       'hour', 'day_of_week', 'day_of_year', 'WCT', 'THI', 'holiday']]
test_df_dl = pd.merge(test_df_dl, building_info, on='building_number', how='left')
test_df_dl = test_df_dl.drop(columns=['solar_power_capacity', 'ess_capacity', 'pcs_capacity'])
display(test_df_dl.head())
display(test_df_dl.dtypes)

for i, j in zip([building_info, train_df_dl, test_df_dl],['building_info', 'train_df_dl', 'test_df_dl']):
    pd.to_pickle(i, f"./processed_data/{j}.pkl")
    print(f"{j}.pkl 저장 완료")

## 분석

### 건물 기본 정보

- 100개의 building_number
- 12가지 building_type
>Other_Buildings       15\
Public          8\
University         8\
Department_Store_and_Outlet     8\
Hospital          8\
Commercial          8\
Apartment         8\
Research_Institute         8\
Knowledge_Industry_Center      8\
Discount_Mart        8\
Hotel_and_Resort      8\
Data_Center       5

- 태양광용량(kW)(solar_power_capacity) : 태양광 발전 설비가 설치된 건물 36개소
> 태양광을 사용하여 전력을 생산할 수 있는 능력


- ESS 시스템 : ESS 시스템 베터리 + PCS(Power Conversion System)이 설치된 건물 5개소
    - ESS 저장용량(kWh): (ESS 시스템 베터리 용량)
>생산된 전기에너지를 저장하여 전력 이 필요한 시기에 사용할 수 있게 하는 에너지 솔루션
    - PCS(Power Conversion System): ESS 내 전력변환장치 
> PCS는 저장된 에너지를 효율적으로 변환하고 제어하는 역할\
PCS의 용량은 ESS에서 저장된 에너지를 공급할 수 있는 능력을 나타낸다

\* 태양광 발전 설비와 ESS 시스템이 모두 설치된 건물: 2개소

In [30]:
# 건물수
print(f'buildings: {len(building_info.building_number.unique())}개',end='\n\n')

# 건물유형
print(f'building_type {len(building_info.building_type.value_counts())}가지: \n{building_info.building_type.value_counts()}',end='\n\n')

# 태양광 발전 설비가 설치된 건물 36개소 
print(f"태양광 발전 설비가 설치된 건물: {len(building_info[building_info['solar_power_capacity'] != 0])}개소", end='\n\n')

# ESS 시스템이 설치된 건물 
print(f"ESS 시스템이 설치된 건물: {len(building_info[building_info[['ess_capacity', 'pcs_capacity']].sum(axis=1) != 0])}개소", end='\n\n')

# 태양광 발전 설비와 ESS 시스템이 모두 설치된 건물
print(f"태양광 발전 설비와 ESS시스템이 모두 설치된 건물: {len(building_info[(building_info['solar_power_capacity'] != 0) & (building_info[['ess_capacity', 'pcs_capacity']].sum(axis=1) != 0)])}개소", end='\n\n')

### 태양광 발전설비 설치 비율

In [31]:
# 태양광발전설비가 설치되어있는 시설 확인
print(f"태양광 발전설비가 설치되어있는 시설: {len(building_info[building_info['solar_power_capacity'] != 0])}개소")
display(building_info[building_info['solar_power_capacity'] != 0])

# 건물유형별 건물 수
building_type = building_info.value_counts('building_type')
building_type = building_type.reset_index()
building_type = building_type.rename(columns={'count':'buildings'})

# 건물유형별 태양광발전설비가 설치되어있는 건물 수
building_type_solar = building_info[building_info['solar_power_capacity'] != 0].value_counts('building_type')
building_type_solar = building_type_solar.reset_index()
building_type_solar = building_type_solar.rename(columns={'count':'solar_power_facilities'})

# 건물유형별 태양광발전설비 설치 비율
building_type_solar_rate = building_type.merge(building_type_solar, how='outer', on='building_type')
building_type_solar_rate = building_type_solar_rate.replace(np.nan,0)
building_type_solar_rate = building_type_solar_rate.set_index('building_type')
building_type_solar_rate['solar_power_facilities_rate'] = (building_type_solar_rate['solar_power_facilities'] / building_type_solar_rate['buildings'])*100
building_type_solar_rate = building_type_solar_rate.sort_values(['solar_power_facilities_rate'], ascending=False)

# total 행 생성
building_type_solar_rate.loc['total', : ] = building_type_solar_rate.iloc[:,:].sum(axis=0)
building_type_solar_rate = building_type_solar_rate.astype('int')
building_type_solar_rate.iloc[-1:,-1:] = np.nan

display(building_type_solar_rate)

In [32]:
plt.figure(figsize=(12,5))
plt.bar(building_type_solar_rate.index, building_type_solar_rate['solar_power_facilities_rate'])
plt.title('태양광 발전설비 설치율(%)', size = 20, weight='bold')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')
plt.xlabel('건물유형', size = 15)
plt.xticks(rotation=35)
plt.ylabel('태양광 발전설비 설치율(%)', size = 15)
plt.show()

- 대학교, 연구소, 병원 순으로 태양광 발전설비 설치율이 높음
- 아파트, 지식산업센터, 데이터센터에는 태양광 발전설비를 설치하지 않음

### Public 20번 건물의 연면적, 냉방면적 이상치 보정
- Public 20번 건물의 평균 전력소비량은 Public 건물의 평균 전력 소비량의 120% 수준
- 반면 데이터에서 Public 20번 건물의 연면적은 Public 건물의 평균의 약 5000%
- Public 20번 건물의 연면적을 Public 건물의 평균의 120%로 보정

In [33]:
train_df = train_df.merge(building_info[['building_number', 'building_type']], on='building_number')
Public_Group = train_df.groupby('building_type').get_group('Public')

# 20번 건물을 제외한 Public 타입 건물 크룹 생성
Public_Group_drop_20 = Public_Group.drop(Public_Group[Public_Group['building_number']==20].index)
display(Public_Group_drop_20)
# Public 건물의 일별 전력 소비량 평군
Mean_Public_drop_20_Day = Public_Group_drop_20.groupby('day_of_year').power_consumption.mean()

plt.figure(figsize=(10, 3))
plt.plot(Mean_Public_drop_20_Day.index, Mean_Public_drop_20_Day.values)
plt.title('20번 건물을 제외한 Public 건물의 일별 전력사용량 평균', size=20)
plt.show()

# Public 20번 건물의 일별 전력 소비량 평군
Mean_Public_20_Day = Public_Group.groupby(by=['building_number', 'day_of_year']).power_consumption.mean()
Mean_Public_20_Day = Mean_Public_20_Day.reset_index()
Mean_Public_20_Day = Mean_Public_20_Day.groupby('building_number').get_group(20)

plt.figure(figsize=(10, 3))
plt.plot(Mean_Public_20_Day.index, Mean_Public_20_Day.power_consumption)
plt.title('Public 20번 건물의 일별 전력 소비량 평군', size=20)
plt.show()

### 상관관계 분석
- 정도가 크지 않았지만 `power_consumption`와 `solar_radiation`, `temperature`,`WCT` 순으로 높은 상관관계를 보임

In [34]:
# 데이터 프레임 병합
train_df = pd.merge(train_df, building_info, on='building_number', how='left')
temp = train_df.pop('power_consumption')
train_df['power_consumption'] = temp
test_df = pd.merge(test_df, building_info, on='building_number', how='left')
display(train_df.head(), test_df.head())

In [35]:
# 사용하지 않을 컬럼 drop
train_df_corr = train_df.copy()
train_df_corr = train_df_corr.drop(columns=['num_date_time', 'building_number', 'date_time', 'year', 'month', 'day', 'hour', 'day_of_week', 'day_of_year', 'building_type'])
train_df_corr

In [ ]:
# 상관관계 heatmap
plt.figure(figsize=(15,8))
sns.heatmap(data = train_df_corr.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')
plt.title('features 상관관계', size=30)
plt.show()

### 시각화

#### 시간당 최대, 최소 전력사용량
- 100개의 빌딩에 전원을 정상적으로 공급하기 위해서는 최소 368310.21(kWh)이상의 전기를 공급할 수 있는 설비가 필요함
> 2022-08-05(금) 13:00:00에 기간내 시간당 최대 전력사용량으로 368310.21(kWh)을 기록\
 2022-06-07(화) 03:00:00에 기간내 시간당 최소 전력사용량으로 148285.615(kWh)을 기록

In [ ]:
시간당_전력사용량_총합 = train_df.groupby('date_time').power_consumption.sum()
시간당_전력사용량_총합
plt.figure(figsize=(15, 5))
plt.plot(시간당_전력사용량_총합.index, 시간당_전력사용량_총합.values)
plt.title('시간당_전력사용량_총합')
plt.xlabel('시간')
plt.ylabel('전력사용량(kWh)')
plt.show()

print(f"시간당 최대 전력사용량: {max(시간당_전력사용량_총합)}(kWh)")
display(시간당_전력사용량_총합[시간당_전력사용량_총합 == max(시간당_전력사용량_총합)])
print(f"시간당 최소 전력사용량: {min(시간당_전력사용량_총합)}(kWh)")
display(시간당_전력사용량_총합[시간당_전력사용량_총합 == min(시간당_전력사용량_총합)])

#### 요일별 전력사용량 평균
- 화요일에 전력사용량 평균이 가장 높음

In [ ]:
요일별_전력사용량_평균 = train_df.groupby('day_of_week')['power_consumption'].mean()
plt.figure(figsize=(15, 5))
plt.plot(['월', '화', '수', '목', '금', '토', '일'],요일별_전력사용량_평균.values)
plt.title('요일별 전력사용량 평균', size=20)
plt.xlabel('요일')
plt.ylabel('전력_사용량_평균(kWh)')
plt.show()

#### 건물별_요일별_전력사용량_평균
- '건물기타': 'Other_Buildings',
- '공공': 'Public' - 주말 전력 소비 감소
- '대학교': 'University' - 주말 전력 소비 감소
- '데이터센터': 'Data_Center' - 매일 전력 소비 일정
- '백화점및아울렛': 'Department_Store_and_Outlet' - 월요일 전력 소비 감소
- '병원': 'Hospital' - 주말 전력 소비 감소
- '상용': 'Commercial' - 주말 전력 소비 감소
- '아파트': 'Apartment' - 매일 전력 소비 일정
- '연구소': 'Research_Institute' - 주말 전력 소비 감소
- '지식산업센터': 'Knowledge_Industry_Center' - 주말 전력 소비 감소
- '할인마트': 'Discount_Mart' - 일요일 전력 소비 감소
- '호텔및리조트': 'Hotel_and_Resort' - 매일 전력 소비 일정

In [ ]:
건물별_요일별_전력사용량_평균 = train_df.groupby(['building_number','building_type','day_of_week', ])
건물별_요일별_전력사용량_평균 = 건물별_요일별_전력사용량_평균.power_consumption.mean()

건물별_요일별_전력사용량_평균 = 건물별_요일별_전력사용량_평균.reset_index()
building_type_list = building_info.building_type.unique()
요일 = ['월', '화', '수', '목', '금', '토', '일']

건물별_요일별_전력사용량_평균 = 건물별_요일별_전력사용량_평균.groupby('building_number')
건물별_요일별_전력사용량_평균.get_group(1)
for i in building_type_list:
    plt.figure(figsize=(15, 5))
    for j in range(1,101):
        if 건물별_요일별_전력사용량_평균.get_group(j).building_type.unique() == i:
            plt.plot(요일, 건물별_요일별_전력사용량_평균.get_group(j).power_consumption, label=f"{j}번 건물")
    plt.title(f'{i} 요일별 전력사용량 평균', size=20)
    plt.xlabel(f'요일')
    plt.ylabel(f'전력사용량 평균(kWh)')
    plt.legend()
    plt.show()

#### 월별 건물별 전력소비량 평균
- 2022년 8월 27번 건물이 월평균 전력소비량 평균이 가장 높음
- 2022년 8월 66번 건물이 월평균 전력소비량 평균이 가장 낮음

In [ ]:
# 월별 건물별 전량 소비량 평균
gr = train_df.groupby(['year','month','building_number']).power_consumption.mean()
gr = gr.reset_index()
gr = gr.merge(building_info[['building_number', 'total_area']], on='building_number')
gr

for i in [6, 7, 8]:
    D = gr[gr.month==i]
    plt.figure(figsize=(10, 4))
    plt.bar(D.building_number,D.power_consumption)
    plt.title(f'{i}월 건물별_전력_소비량_평균 ')
    plt.xlabel('건물번호')
    plt.ylabel('전력소비량 평균(kWh)')
    plt.show()
display(D[D.power_consumption == max(D.power_consumption)])
display(D[D.power_consumption == min(D.power_consumption)])

#### 월별 건물별 단위 면적당 전력소비량
- 13번 건물의 단위 면적당 전력소비량이 가장 높음
- 20번 건물의 단위 면적당 전력소비량이 가장 낮음

In [ ]:
# 월별 건물별 전량 소비량 평균
gr = train_df.groupby(['year','month','building_number']).power_consumption.sum()
gr = gr.reset_index()
gr = gr.merge(building_info[['building_number', 'total_area']], on='building_number')

for i in [6, 7, 8]:
    D = gr[gr.month==i]
    plt.figure(figsize=(10, 4))
    plt.bar(D.building_number,D.power_consumption/D.total_area)
    plt.title(f'{i}월 건물별_단위면적당_전량소비량 ')
    plt.xlabel('건물번호')
    plt.ylabel('단위면적당_전량소비량(kWh/m2)')
    plt.show()
    
display(building_info[12:13])
display(building_info[19:20])

#### 건물별 시간당 전력소비량 빈도
- 사례의 100개 건물의 시간당 전력 사용량 약 92.3%는 5000kWh 이하이다.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train_df['power_consumption'], bins=100, kde=True)
plt.title('건물별 시간당 전력소비량 빈도')
plt.xlabel('시간당 전력소비량(kWh)')
plt.show()

In [ ]:
train_df[train_df['power_consumption']==max(train_df['power_consumption'])]

In [ ]:
len(train_df[train_df['power_consumption']<=5000])/len(train_df)

#### 단위 기간별 평균 전력사용량

In [ ]:
# 년기준 월평균 소비전력
mean_power_by_month = train_df.groupby('month')['power_consumption'].mean()

# 일기준 시간 평균 소비전력
mean_power_by_hour = train_df.groupby('hour')['power_consumption'].mean()

# 월기준 시간 평균 소비전력
mean_power_by_day = train_df.groupby('day')['power_consumption'].mean()

# 년기준 일 평균 소비전력
mean_power_by_day_of_year = train_df.groupby('day_of_year')['power_consumption'].mean()

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(10, 18))

# Plot mean power consumption by hour of the day
axs[0].plot(mean_power_by_hour.index,mean_power_by_hour.values)
axs[0].set_title('하루 시간별 전력소비량 평균', size=20)
axs[0].set_xlabel('시간')
axs[0].set_ylabel('하루 시간별 전력 소비량(kWh)')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')

# Plot mean power consumption by day of the month
axs[1].plot(mean_power_by_day.index,mean_power_by_day.values*24)
axs[1].set_title('월간 일별 전력소비량 평균', size=20)
axs[1].set_xlabel('일')
axs[1].set_ylabel('월간 일별 전력소비량(kWh)')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')

# 년기준 월평균 소비전력
axs[2].bar(mean_power_by_month.index,mean_power_by_month.values*24*30)
axs[2].set_title('월별 전력소비량 평균', size=20)
axs[2].set_xlabel('월')
axs[2].set_ylabel('월별 전력소비량(kWh)')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')

# Plot mean power consumption by hour of the day
axs[3].plot(mean_power_by_day_of_year.index,mean_power_by_day_of_year.values*24)
axs[3].set_title('연간 하루 전력소비량 평균', size=20)
axs[3].set_xlabel('시간')
axs[3].set_ylabel('하루 시간별 전력 소비량 평균(kWh)')

plt.tight_layout()
plt.show()